In [1]:
from datetime import datetime, timedelta,date
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

from __future__ import division
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report,confusion_matrix




In [2]:
df_expe = pd.read_csv("DATOS_TFM_ANALISISv2.csv",sep =";", encoding='latin',decimal=",",thousands=".")



/tmp/ipykernel_4083/792331613.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_expe = pd.read_csv("DATOS_TFM_ANALISISv2.csv",sep =";", encoding='latin',decimal=",",thousands=".")


In [3]:
df_expe.head(10)

,Fecha,expedicion,Nº Cliente,Departamento,Tipo Portes,C.P. Origen,Pob. Origen,C.P. Destino,Pob. Destino,País Destino,Ámbito,Servicio,Envase,Kilos,Bultos,Excesos,Situación,Fecha Ent.,Receptor,Expe_km_entrega
0,04/01/2022,1,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,DAC,BAG,1.0,1,0,OK,05/01/2022,RECEPTOR,0.0
1,12/01/2022,2,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,DAC,BAG,1.0,1,0,OK,13/01/2022,RECEPTOR,0.0
2,14/01/2022,3,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,DAC,BAG,1.0,1,0,OK,17/01/2022,RECEPTOR,0.0
3,20/01/2022,4,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,DAC,BAG,1.0,1,0,OK,21/01/2022,RECEPTOR,0.0
4,24/01/2022,5,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,NX19,PAQ,9.0,2,2,OK,25/01/2022,RECEPTOR,0.0
5,31/01/2022,6,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,DAC,BAG,1.0,1,0,OK,01/02/2022,RECEPTOR,0.0
6,02/02/2022,7,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,NAC,NX19,PAQ,16.0,2,1,OK,03/02/2022,RECEPTOR,0.0
7,15/02/2022,8,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,NAC,DAC,BAG,1.0,1,0,OK,16/02/2022,RECEPTOR,0.0
8,17/02/2022,9,2,SHIPMENT,T,NaN,NaN,6422PH,HEERLEN,NL-HOLANDA,Z-1,EUT,MUE,0.7,1,0,OK,18/02/2022,SELLO,0.0
9,17/02/2022,10,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,NAC,NX19,PAQ,6.0,1,1,OK,18/02/2022,RECEPTOR,0.0


In [4]:
df_expe.shape

(384692, 20)

In [5]:
df_expe.describe()

,expedicion,Nº Cliente,C.P. Origen,Kilos,Bultos,Excesos,Expe_km_entrega
count,384692.000000,384692.000000,3.782110e+05,384692.000000,384692.000000,384692.000000,384692.000000
mean,192346.500000,119.167898,1.557633e+04,2.022523,1.136694,0.062141,9.946906
std,111051.159214,274.190789,1.358420e+05,10.814384,1.239855,0.366659,29.710249
min,1.000000,0.000000,1.004000e+03,0.300000,1.000000,0.000000,0.000000
25%,96173.750000,14.000000,8.097000e+03,1.000000,1.000000,0.000000,0.000000
50%,192346.500000,50.000000,8.097000e+03,1.000000,1.000000,0.000000,0.000000
75%,288519.250000,204.000000,8.208000e+03,1.000000,1.000000,0.000000,0.000000
max,384692.000000,99989.000000,9.900106e+06,2742.000000,107.000000,30.000000,506.000000


In [6]:
df_expe.columns

Index(['Fecha', 'expedicion ', 'Nº Cliente', 'Departamento', 'Tipo Portes',
       'C.P. Origen', 'Pob. Origen', 'C.P. Destino', 'Pob. Destino',
       'País Destino', 'Ámbito', 'Servicio', 'Envase', 'Kilos', 'Bultos',
       'Excesos', 'Situación', 'Fecha Ent.', 'Receptor', 'Expe_km_entrega'],
      dtype='object')

In [6]:
print(df_expe['Servicio'].unique())

['DAC' 'NX19' 'EUT' 'PPK' 'ENX' 'NX12' 'BMR' '830H' 'PB1' 'P.U.' 'NX10'
 'SAB' 'I-P' '24H' 'INT2' '48H' 'CMR' 'EUE' 'AER' 'PPE' 'EXS' 'P' 'INT1']


In [7]:
df_expe["service"] = df_expe.Servicio.replace({'DAC': "OT", 'NX19':"19H", 'EUT':"OT", 'PPK':'PPK', 'ENX':"ECO", 'NX12':"PRE", 'BMR':"OT", '830H':"PRE", 'PB1':"OT", 'P.U.':"OT", 'NX10': "PRE",
 'SAB':"OT", 'I-P':"OT", '24H':"CAN", 'INT2':"OT", '48H':"CAN", 'CMR':"OT", 'EUE':"OT", 'AER':"OT", 'PPE':"OT", 'EXS':"ECO", 'P':"OT", 'INT1':"OT"})

df_expe.head(10)

,Fecha,expedicion,Nº Cliente,Departamento,Tipo Portes,C.P. Origen,Pob. Origen,C.P. Destino,Pob. Destino,País Destino,...,Servicio,Envase,Kilos,Bultos,Excesos,Situación,Fecha Ent.,Receptor,Expe_km_entrega,service
0,04/01/2022,1,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,DAC,BAG,1.0,1,0,OK,05/01/2022,RECEPTOR,0.0,OT
1,12/01/2022,2,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,DAC,BAG,1.0,1,0,OK,13/01/2022,RECEPTOR,0.0,OT
2,14/01/2022,3,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,DAC,BAG,1.0,1,0,OK,17/01/2022,RECEPTOR,0.0,OT
3,20/01/2022,4,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,DAC,BAG,1.0,1,0,OK,21/01/2022,RECEPTOR,0.0,OT
4,24/01/2022,5,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,NX19,PAQ,9.0,2,2,OK,25/01/2022,RECEPTOR,0.0,19H
5,31/01/2022,6,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,DAC,BAG,1.0,1,0,OK,01/02/2022,RECEPTOR,0.0,OT
6,02/02/2022,7,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,NX19,PAQ,16.0,2,1,OK,03/02/2022,RECEPTOR,0.0,19H
7,15/02/2022,8,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,DAC,BAG,1.0,1,0,OK,16/02/2022,RECEPTOR,0.0,OT
8,17/02/2022,9,2,SHIPMENT,T,NaN,NaN,6422PH,HEERLEN,NL-HOLANDA,...,EUT,MUE,0.7,1,0,OK,18/02/2022,SELLO,0.0,OT
9,17/02/2022,10,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,NX19,PAQ,6.0,1,1,OK,18/02/2022,RECEPTOR,0.0,19H


In [8]:
df_expe["service_value"] = df_expe.service.replace({'ECO': 1.0, '19H':1.9, 'OT':1.4, 'PPK':1.1, 'PRE':3.0, 'CAN':1.5})

df_expe.head(10)

,Fecha,expedicion,Nº Cliente,Departamento,Tipo Portes,C.P. Origen,Pob. Origen,C.P. Destino,Pob. Destino,País Destino,...,Envase,Kilos,Bultos,Excesos,Situación,Fecha Ent.,Receptor,Expe_km_entrega,service,service_value
0,04/01/2022,1,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,BAG,1.0,1,0,OK,05/01/2022,RECEPTOR,0.0,OT,1.4
1,12/01/2022,2,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,BAG,1.0,1,0,OK,13/01/2022,RECEPTOR,0.0,OT,1.4
2,14/01/2022,3,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,BAG,1.0,1,0,OK,17/01/2022,RECEPTOR,0.0,OT,1.4
3,20/01/2022,4,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,BAG,1.0,1,0,OK,21/01/2022,RECEPTOR,0.0,OT,1.4
4,24/01/2022,5,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,PAQ,9.0,2,2,OK,25/01/2022,RECEPTOR,0.0,19H,1.9
5,31/01/2022,6,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,BAG,1.0,1,0,OK,01/02/2022,RECEPTOR,0.0,OT,1.4
6,02/02/2022,7,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,PAQ,16.0,2,1,OK,03/02/2022,RECEPTOR,0.0,19H,1.9
7,15/02/2022,8,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,BAG,1.0,1,0,OK,16/02/2022,RECEPTOR,0.0,OT,1.4
8,17/02/2022,9,2,SHIPMENT,T,NaN,NaN,6422PH,HEERLEN,NL-HOLANDA,...,MUE,0.7,1,0,OK,18/02/2022,SELLO,0.0,OT,1.4
9,17/02/2022,10,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,PAQ,6.0,1,1,OK,18/02/2022,RECEPTOR,0.0,19H,1.9


In [9]:
for x in df_expe["Kilos"]:
    if x >=5:
        df_expe["Kilos_value"] = 3
    elif x >=10:
        df_expe["Kilos_value"] = 2
    elif x >=20:
        df_expe["Kilos_value"] = 1
    else:
        df_expe["Kilos_value"] = 0



df_expe.head(10)

#np.where



,Fecha,expedicion,Nº Cliente,Departamento,Tipo Portes,C.P. Origen,Pob. Origen,C.P. Destino,Pob. Destino,País Destino,...,Kilos,Bultos,Excesos,Situación,Fecha Ent.,Receptor,Expe_km_entrega,service,service_value,Kilos_value
0,04/01/2022,1,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,05/01/2022,RECEPTOR,0.0,OT,1.4,3
1,12/01/2022,2,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,13/01/2022,RECEPTOR,0.0,OT,1.4,3
2,14/01/2022,3,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,17/01/2022,RECEPTOR,0.0,OT,1.4,3
3,20/01/2022,4,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,21/01/2022,RECEPTOR,0.0,OT,1.4,3
4,24/01/2022,5,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,9.0,2,2,OK,25/01/2022,RECEPTOR,0.0,19H,1.9,3
5,31/01/2022,6,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,01/02/2022,RECEPTOR,0.0,OT,1.4,3
6,02/02/2022,7,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,16.0,2,1,OK,03/02/2022,RECEPTOR,0.0,19H,1.9,3
7,15/02/2022,8,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,16/02/2022,RECEPTOR,0.0,OT,1.4,3
8,17/02/2022,9,2,SHIPMENT,T,NaN,NaN,6422PH,HEERLEN,NL-HOLANDA,...,0.7,1,0,OK,18/02/2022,SELLO,0.0,OT,1.4,3
9,17/02/2022,10,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,6.0,1,1,OK,18/02/2022,RECEPTOR,0.0,19H,1.9,3


In [10]:
df_muestra = df_expe.head(20)
df_muestra

,Fecha,expedicion,Nº Cliente,Departamento,Tipo Portes,C.P. Origen,Pob. Origen,C.P. Destino,Pob. Destino,País Destino,...,Kilos,Bultos,Excesos,Situación,Fecha Ent.,Receptor,Expe_km_entrega,service,service_value,Kilos_value
0,04/01/2022,1,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,05/01/2022,RECEPTOR,0.0,OT,1.4,3
1,12/01/2022,2,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,13/01/2022,RECEPTOR,0.0,OT,1.4,3
2,14/01/2022,3,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,17/01/2022,RECEPTOR,0.0,OT,1.4,3
3,20/01/2022,4,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,21/01/2022,RECEPTOR,0.0,OT,1.4,3
4,24/01/2022,5,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,9.0,2,2,OK,25/01/2022,RECEPTOR,0.0,19H,1.9,3
5,31/01/2022,6,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,01/02/2022,RECEPTOR,0.0,OT,1.4,3
6,02/02/2022,7,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,16.0,2,1,OK,03/02/2022,RECEPTOR,0.0,19H,1.9,3
7,15/02/2022,8,14,NaN,D,1004.0,VITORIA,08097,BARCELONA (ABONADO ),ES-ESPAÑA,...,1.0,1,0,OK,16/02/2022,RECEPTOR,0.0,OT,1.4,3
8,17/02/2022,9,2,SHIPMENT,T,NaN,NaN,6422PH,HEERLEN,NL-HOLANDA,...,0.7,1,0,OK,18/02/2022,SELLO,0.0,OT,1.4,3
9,17/02/2022,10,14,NaN,D,1004.0,VITORIA,08097,SPEEDLINK,ES-ESPAÑA,...,6.0,1,1,OK,18/02/2022,RECEPTOR,0.0,19H,1.9,3


In [34]:
df_muestra.groupby(['Nº Cliente','service']).agg({'expedicion '.count()})
#df.groupby(['A', 'B'])['C'].count()

#df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})
#['Fecha', 'expedicion ', 'Nº Cliente', 'Departamento', 'Tipo Portes',
#      'C.P. Origen', 'Pob. Origen', 'C.P. Destino', 'Pob. Destino',
#       'País Destino', 'Ámbito', 'Servicio', 'Envase', 'Kilos', 'Bultos',
#       'Excesos', 'Situación', 'Fecha Ent.', 'Receptor', 'Expe_km_entrega'],
#      dtype='object')
print(df_muestra)

TypeError: count() takes at least 1 argument (0 given)

In [32]:
df_muestra.groupby(['Nº Cliente','service']).['expedicion '].count()
#df.groupby(['A', 'B'])['C'].count()

df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})
#['Fecha', 'expedicion ', 'Nº Cliente', 'Departamento', 'Tipo Portes',
#      'C.P. Origen', 'Pob. Origen', 'C.P. Destino', 'Pob. Destino',
#       'País Destino', 'Ámbito', 'Servicio', 'Envase', 'Kilos', 'Bultos',
#       'Excesos', 'Situación', 'Fecha Ent.', 'Receptor', 'Expe_km_entrega'],
#      dtype='object')
print(df_muestra)

         Fecha  expedicion   Nº Cliente Departamento Tipo Portes  C.P. Origen  \
0   04/01/2022            1          14          NaN           D       1004.0   
1   12/01/2022            2          14          NaN           D       1004.0   
2   14/01/2022            3          14          NaN           D       1004.0   
3   20/01/2022            4          14          NaN           D       1004.0   
4   24/01/2022            5          14          NaN           D       1004.0   
5   31/01/2022            6          14          NaN           D       1004.0   
6   02/02/2022            7          14          NaN           D       1004.0   
7   15/02/2022            8          14          NaN           D       1004.0   
8   17/02/2022            9           2     SHIPMENT           T          NaN   
9   17/02/2022           10          14          NaN           D       1004.0   
10  21/02/2022           11          14          NaN           D       1004.0   
11  28/02/2022           12 

In [13]:
df_expeuser=pd.DataFrame(df_expe['Nº Cliente'].unique())
df_expeuser.columns=['Nº Cliente']
df_expeuser.head(10)

,Nº Cliente
0,14
1,2
2,50
3,300
4,60
5,206
6,301
7,204
8,202
9,1


In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(df_expe)
profile

In [ ]:
df_expeuser=pd.DataFrame(df_expe['Nº Cliente'].unique())
df_expeuser.columns=['Nº Cliente']

In [ ]:
df_expeuser.head(10)

In [ ]:
df_expe["Envase_num"]= df['Envase']

In [ ]:
df_expe.Envase_num.head(10)

In [ ]:
df_expe["Envase_num"] = df_expe.Envase.replace({'BAG': 1, 'PAQ': 2, 'MUE': 3, 'DOC': 3, 'DOCS': 3})
df_expe.head(10)

In [ ]:
num_envase = {'BAG':1, 'PAQ':2 'MUE':3, 'DOC':3, 'DOCS':3}